# MISO Weather-Stress Heatmap MVP

## Overview
This notebook generates weather-related grid stress risk assessments for the MISO footprint using short-term weather forecasts and infrastructure data. The system produces interactive heat maps for multiple forecast horizons (12h, 24h, 36h, 48h) with transparent, explainable risk scoring.

## Key Features
- **Multi-source weather data**: NOAA/NWS primary, Open-Meteo fallback
- **Infrastructure integration**: EIA capacity data, Census population, transmission density
- **Transparent risk scoring**: Configurable weights, documented methodology
- **Interactive visualization**: Folium maps with tooltips and layer controls
- **Confidence metrics**: Data coverage and forecast horizon-based confidence
- **Reproducible analysis**: Fixed random seeds, comprehensive logging

## Methodology
Risk = zscore(α×Hazard + β×Exposure + γ×Vulnerability)
- **Hazard**: Weather stress (thermal, wind, precipitation, storms)
- **Exposure**: Population density and load factors
- **Vulnerability**: Renewable share, transmission scarcity, outage flags

---

## Configuration and Setup

In [ ]:
# Core imports
import os
import sys
import yaml
import json
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Union
from dataclasses import dataclass
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')

# Data processing
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import unary_union
import h3

# Spatial and coordinate systems
import pyproj
from pyproj import CRS, Transformer
import rasterio
from rasterio.features import shapes
from rasterio.transform import from_bounds

# Weather and web APIs
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time

# Visualization
import folium
from folium import plugins
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical analysis
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

print("✓ All imports successful")
print(f"Notebook initialized at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Configuration Manager
class ConfigurationManager:
    """Centralized configuration handling and runtime mode management"""
    
    def __init__(self):
        self.config = self._load_default_config()
        self.runtime_mode = "demo"  # Default to demo mode
        self.random_seed = 42
        self._setup_logging()
        self._set_random_seed()
    
    def _load_default_config(self) -> Dict:
        """Load default configuration with all required parameters"""
        return {
            'runtime': {
                'mode': 'demo',
                'horizons_h': [12, 24, 36, 48],
                'crs': 'EPSG:4326',
                'random_seed': 42,
                'hex_size_km': 40,
                'api_timeout': 30,
                'max_retries': 3
            },
            'weights': {
                'hazard': {
                    'thermal': 0.3,
                    'wind': 0.3,
                    'precip': 0.25,
                    'storm': 0.15
                },
                'exposure': {
                    'pop': 0.7,
                    'load': 0.3
                },
                'vulnerability': {
                    'renew_share': 0.6,
                    'tx_scarcity': 0.3,
                    'outage': 0.1
                },
                'blend': {
                    'alpha': 0.5,  # hazard weight
                    'beta': 0.3,   # exposure weight
                    'gamma': 0.2   # vulnerability weight
                }
            },
            'thresholds': {
                'thermal': {
                    'heat_low': 85,   # °F
                    'heat_high': 100,
                    'cold_low': 10,
                    'cold_high': 0
                },
                'wind': {
                    'gust_low': 20,   # mph
                    'gust_high': 50,
                    'sustained_threshold': 30
                },
                'precip': {
                    'rain_heavy': 10,  # mm/h
                    'snow_heavy': 5,   # cm/h
                    'ice_threshold': 0  # any ice = max score
                },
                'confidence': {
                    'base_confidence': 0.9,
                    'horizon_decay': 0.05,  # confidence decrease per 12h
                    'coverage_threshold': 0.8
                }
            },
            'data_sources': {
                'noaa_base_url': 'https://api.weather.gov',
                'openmeteo_base_url': 'https://api.open-meteo.com/v1/forecast',
                'eia_capacity_url': 'https://www.eia.gov/electricity/data/eia860/',
                'census_api_base': 'https://api.census.gov/data'
            },
            'output': {
                'map_width': 1200,
                'map_height': 800,
                'color_scale': 'YlOrRd',
                'export_formats': ['html', 'png', 'csv', 'md']
            }
        }
    
    def load_config_from_yaml(self, yaml_string: str) -> None:
        """Load configuration from YAML string and merge with defaults"""
        try:
            user_config = yaml.safe_load(yaml_string)
            if user_config:
                self._deep_merge(self.config, user_config)
                self.validate_config()
                logging.info("Configuration updated from YAML")
        except yaml.YAMLError as e:
            logging.error(f"YAML parsing error: {e}")
            raise
    
    def _deep_merge(self, base_dict: Dict, update_dict: Dict) -> None:
        """Recursively merge update_dict into base_dict"""
        for key, value in update_dict.items():
            if key in base_dict and isinstance(base_dict[key], dict) and isinstance(value, dict):
                self._deep_merge(base_dict[key], value)
            else:
                base_dict[key] = value
    
    def set_runtime_mode(self, mode: str) -> None:
        """Switch between demo and live modes"""
        if mode not in ['demo', 'live']:
            raise ValueError("Mode must be 'demo' or 'live'")
        
        self.runtime_mode = mode
        self.config['runtime']['mode'] = mode
        logging.info(f"Runtime mode set to: {mode}")
    
    def validate_config(self) -> bool:
        """Validate configuration completeness and consistency"""
        required_sections = ['runtime', 'weights', 'thresholds', 'data_sources', 'output']
        
        for section in required_sections:
            if section not in self.config:
                raise ValueError(f"Missing required config section: {section}")
        
        # Validate weight sums
        hazard_sum = sum(self.config['weights']['hazard'].values())
        if not (0.95 <= hazard_sum <= 1.05):  # Allow small floating point errors
            logging.warning(f"Hazard weights sum to {hazard_sum:.3f}, not 1.0")
        
        blend_sum = sum(self.config['weights']['blend'].values())
        if not (0.95 <= blend_sum <= 1.05):
            logging.warning(f"Blend weights sum to {blend_sum:.3f}, not 1.0")
        
        return True
    
    def _setup_logging(self) -> None:
        """Configure logging system"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.StreamHandler(),
                logging.FileHandler('output/miso_heatmap.log')
            ]
        )
    
    def _set_random_seed(self) -> None:
        """Set random seed for reproducibility"""
        self.random_seed = self.config['runtime']['random_seed']
        np.random.seed(self.random_seed)
        logging.info(f"Random seed set to: {self.random_seed}")
    
    def get_config(self) -> Dict:
        """Return current configuration"""
        return self.config.copy()
    
    def get_runtime_mode(self) -> str:
        """Return current runtime mode"""
        return self.runtime_mode
    
    def log_config_summary(self) -> None:
        """Log key configuration parameters"""
        config = self.config
        logging.info("=== Configuration Summary ===")
        logging.info(f"Runtime mode: {self.runtime_mode}")
        logging.info(f"Forecast horizons: {config['runtime']['horizons_h']}")
        logging.info(f"Coordinate system: {config['runtime']['crs']}")
        logging.info(f"Hex grid size: {config['runtime']['hex_size_km']} km")
        logging.info(f"Random seed: {self.random_seed}")
        logging.info(f"Blend weights - α:{config['weights']['blend']['alpha']}, β:{config['weights']['blend']['beta']}, γ:{config['weights']['blend']['gamma']}")

# Initialize configuration manager
config_manager = ConfigurationManager()
print("✓ Configuration manager initialized")

### User Configuration (YAML)
Modify the configuration below to customize weights, thresholds, and runtime parameters:

In [ ]:
# User-configurable YAML configuration
# Modify values below to customize the analysis

user_config_yaml = """
runtime:
  mode: "demo"  # Change to "live" for real API calls
  horizons_h: [12, 24, 36, 48]
  hex_size_km: 40

weights:
  # Hazard component weights (should sum to 1.0)
  hazard:
    thermal: 0.30   # Temperature extremes (heat/cold)
    wind: 0.30      # Wind gusts and sustained winds
    precip: 0.25    # Rain, snow, ice precipitation
    storm: 0.15     # Combined storm conditions
  
  # Exposure component weights
  exposure:
    pop: 0.70       # Population density weight
    load: 0.30      # Load center weight (if available)
  
  # Vulnerability component weights
  vulnerability:
    renew_share: 0.60    # Renewable generation share
    tx_scarcity: 0.30    # Transmission line density
    outage: 0.10         # Historical outage flags
  
  # Final risk blend weights (should sum to 1.0)
  blend:
    alpha: 0.50     # Hazard weight in final risk
    beta: 0.30      # Exposure weight in final risk
    gamma: 0.20     # Vulnerability weight in final risk

thresholds:
  # Temperature stress thresholds (°F)
  thermal:
    heat_low: 85    # Heat stress starts (0 score)
    heat_high: 100  # Maximum heat stress (1 score)
    cold_low: 10    # Cold stress starts (0 score)
    cold_high: 0    # Maximum cold stress (1 score)
  
  # Wind stress thresholds (mph)
  wind:
    gust_low: 20    # Wind stress starts (0 score)
    gust_high: 50   # Maximum wind stress (1 score)
    sustained_threshold: 30  # Sustained wind bonus threshold
  
  # Precipitation stress thresholds
  precip:
    rain_heavy: 10  # Heavy rain threshold (mm/h)
    snow_heavy: 5   # Heavy snow threshold (cm/h)
    ice_threshold: 0  # Any ice = maximum score
"""

# Load user configuration
config_manager.load_config_from_yaml(user_config_yaml)
config_manager.log_config_summary()

print("✓ User configuration loaded and validated")

### Runtime Mode Selection

In [ ]:
# Runtime mode configuration
# Set to "demo" for cached/sample data, "live" for real API calls

RUN_MODE = "demo"  # Change to "live" for production use

config_manager.set_runtime_mode(RUN_MODE)

# Display current configuration
current_config = config_manager.get_config()

print(f"🔧 Runtime Mode: {config_manager.get_runtime_mode().upper()}")
print(f"📊 Forecast Horizons: {current_config['runtime']['horizons_h']} hours")
print(f"🌐 Coordinate System: {current_config['runtime']['crs']}")
print(f"🔢 Random Seed: {current_config['runtime']['random_seed']}")
print(f"📐 Hex Grid Size: {current_config['runtime']['hex_size_km']} km")

if RUN_MODE == "demo":
    print("\n⚠️  DEMO MODE: Using cached/sample data for development")
    print("   - No live API calls will be made")
    print("   - Results are for demonstration purposes only")
else:
    print("\n🌐 LIVE MODE: Making real API calls")
    print("   - Fetching current weather forecasts")
    print("   - Results reflect real-time conditions")

print("\n✓ Runtime configuration complete")

### Directory Structure Verification

In [ ]:
# Verify and create required directory structure
required_dirs = [
    'data/raw',
    'data/processed', 
    'output'
]

print("📁 Directory Structure:")
for directory in required_dirs:
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
        print(f"   ✓ Created: {directory}/")
    else:
        print(f"   ✓ Exists: {directory}/")

# Log directory creation
logging.info("Directory structure verified and created")
logging.info(f"Working directory: {os.getcwd()}")

print("\n✓ Directory structure ready")

### System Information and Dependencies

In [ ]:
# System information and dependency verification
import platform
import subprocess

print("🖥️  System Information:")
print(f"   Platform: {platform.system()} {platform.release()}")
print(f"   Python: {platform.python_version()}")
print(f"   Architecture: {platform.machine()}")

print("\n📦 Key Dependencies:")
key_packages = {
    'numpy': np.__version__,
    'pandas': pd.__version__,
    'geopandas': gpd.__version__,
    'folium': folium.__version__,
    'requests': requests.__version__
}

for package, version in key_packages.items():
    print(f"   {package}: {version}")

# Log system information
logging.info(f"System: {platform.system()} {platform.release()}")
logging.info(f"Python: {platform.python_version()}")
for package, version in key_packages.items():
    logging.info(f"{package}: {version}")

print("\n✓ System verification complete")
print("\n" + "="*60)
print("🚀 MISO Weather-Stress Heatmap MVP Ready")
print("   Configuration loaded and validated")
print("   Directory structure created")
print("   Dependencies verified")
print("   Logging system active")
print("="*60)